In [6]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 데이터 적재와 분할
cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)
# 훈련 데이터의 최솟값, 최댓값을 계산합니다.
scaler = MinMaxScaler().fit(X_train)

In [9]:
# 훈련 데이터의 스케일을 조정합니다.
X_train_scaled = scaler.transform(X_train)

svm = SVC()
# 스케일 조정된 훈련데이터에 SVM을 학습시킵니다.
svm.fit(X_train_scaled, y_train)
# 테스트 데이터의 스케일을 조정하고 점수를 계산합니다.
X_test_scaled = scaler.transform(X_test)
print("테스트 점수 : {:.2f}".format(svm.score(X_test_scaled, y_test)))

테스트 점수 : 0.97


## 파이프라인 구축하기
- Pipeline은 여러 처리 단계를 하나의 sklearn 추정기 형태로 묶어주는 파이썬 클래스

In [10]:
# MinMaxScaler로 데이터 스케일을 조정, SVM 모델을 훌련시키는 워크플로를 표현
from sklearn.pipeline import Pipeline
pipe = Pipeline([("scaler",MinMaxScaler()),("svm",SVC())])

In [11]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', MinMaxScaler()), ('svm', SVC())])

In [15]:
print("테스트 점수 : {:.2f}".format(pipe.score(X_test,y_test)))

테스트 점수 : 0.97


## 그리드 서치에 파이프라인 적용하기

In [14]:
from sklearn.model_selection import GridSearchCV
# 딕셔너리의 키를 __ 를 붙여서 만들어야 합니다.
param_grid = {'svm__C' : [0.001, 0.01, 0.1, 1, 10, 100],
             'svm__gamma' : [0.001,0.01,0.1,1,10,100]}

In [17]:
grid = GridSearchCV(pipe, param_grid=param_grid,cv=5)
grid.fit(X_train, y_train)
print("최상의 교차 검증 정확도 : {:.2f}".format(grid.best_score_))
print("테스트 세트 점수 : {:.2f}".format(grid.score(X_test,y_test)))
print("최적의 매개변수 : ", grid.best_params_)

최상의 교차 검증 정확도 : 0.98
테스트 세트 점수 : 0.97
최적의 매개변수 :  {'svm__C': 1, 'svm__gamma': 1}


In [18]:
from sklearn.pipeline import make_pipeline
pipe_short = make_pipeline(MinMaxScaler(),SVC(C=100))